## LSTM model selection
To determine the appropriate architecture and hyper parameters for the LSTMs we drew 200 samples from a randomized grid of possibilities for each of the five folds. The following values defined the grid: whether or not to use a bidirectional LSTM; the number of LSTM layers (1 or 2) for the non-bidirectional LSTMs; whether or not to include a penultimate dense layer (after the LSTM and prior to the prediction); the number of neurons for each layer (32, 64, 128 or 256) and; the dropout rates for the LSTM layers and the penultimate dense layer (0.1, 0.2 or 0.5).

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [3]:
import re
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import glob

In [4]:
from keras.layers import Input, LSTM, Dense, Dropout, Bidirectional
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from keras import backend as K
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from datetime import datetime

In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

### Settings

In [6]:
fold = 'fold5'
fit_method = 'regress' # clasify or regress

train_ts = np.load('/scratch-shared/phil/LNP/LNP_data_09/train_ts_' + fit_method + '_' + fold + '.npy')
train_y = np.load('/scratch-shared/phil/LNP/LNP_data_09/train_cell_gfp_' + fit_method + '_' + fold + '.npy')
train_ids = np.load('/scratch-shared/phil/LNP/LNP_data_09/train_cell_ids_' + fit_method + '_' + fold + '.npy')

train_index = []
valid_index = []

for i in range(len(train_ids)):
    s0 = train_ids[i].split('train/')
    s1 = s0[1].split('_')[0]
    if s1 == fold:
        valid_index.append(i)
    else:
        train_index.append(i)

train_ts_train = train_ts[train_index]
train_y_train = train_y[train_index]
train_ts_valid = train_ts[valid_index]
train_y_valid = train_y[valid_index]

n_epochs = 20
batch_size = 32
n_time = 20
n_latent = 32
lr = 0.0001

col_names = ['layer_0', 'layer_1', 'layer_2', 'rec_do', 'main_do', 'layer_end', 'end_do', 'val_loss']
# save_best = keras.callbacks.ModelCheckpoint('best.weights', monitor='val_loss', verbose=0, save_best_only=True)

num_layers = [0, 1, 2] # 0 for bidirectional 
layer_filters = [32, 64, 128, 256]
drop_out_rates = [0.1, 0.2, 0.5]

# getting class weights for when in classification mode
if fit_method == 'classify':
    class_gfp = train_y_train.astype('int64')
    weights = class_weight.compute_class_weight('balanced', np.unique(class_gfp), class_gfp)
    print('weights = ' + str(weights))

### Grid search

In [7]:
for grid_rep in range(200):
    layer_0 = 0
    layer_1 = 0
    layer_2 = 0
    rec_do = np.random.choice(drop_out_rates) # recurrent dropout in LSTM
    main_do = np.random.choice(drop_out_rates) # main dropout in LSTM
    layer_end = 0 # if have an aditional layer after LSTM
    end_do = 0
    n_layers = np.random.choice(num_layers)

    input_ts = Input(shape=(n_time, n_latent))
    x = input_ts

    if n_layers == 0:
        layer_0 = np.random.choice(layer_filters)
        x = Bidirectional(LSTM(layer_0, dropout=main_do, recurrent_dropout=rec_do,
                               input_shape=(n_time, n_latent)))(x)
    else:
        layer_1 = np.random.choice(layer_filters)
        if np.random.random() > 0.5:
            x = LSTM(layer_1, dropout=main_do, recurrent_dropout=rec_do,
                     return_sequences=True,
                     input_shape=(n_time, n_latent))(x)

            layer_2 = np.random.choice(layer_filters)
            x = LSTM(layer_2, dropout=main_do, recurrent_dropout=rec_do)(x)
        else:
            x = LSTM(layer_1, dropout=main_do, recurrent_dropout=rec_do,
                     input_shape=(n_time, n_latent))(x)

    if np.random.random() > 0.5:
        layer_end = np.random.choice(layer_filters)
        end_do = np.random.choice(drop_out_rates)
        x = Dense(layer_end, activation='relu')(x)
        x = Dropout(end_do)(x)

    if fit_method == 'classify': 
        x = Dense(1, activation='sigmoid')(x)
    else:
        x = Dense(1)(x)

    model = Model(input_ts, x)

    if fit_method == 'classify':
        model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['acc'])
        history = model.fit(
            train_ts_train, train_y_train,
            steps_per_epoch=len(train_ts_train) // batch_size,
            epochs = n_epochs,
            shuffle = True,
            validation_data = (train_ts_valid, train_y_valid),
            validation_steps = len(train_ts_valid) // batch_size,
            # callbacks = [save_best],
            class_weight = weights,
            verbose = 0
        )
    else:
        model.compile(optimizer=Adam(lr=lr), loss='mse')
        history = model.fit(
            train_ts_train, train_y_train,
            steps_per_epoch=len(train_ts_train) // batch_size,
            epochs = n_epochs,
            shuffle = True,
            validation_data = (train_ts_valid, train_y_valid),
            validation_steps = len(train_ts_valid) // batch_size,
            # callbacks = [save_best],
            verbose = 0
        )
    
    res = np.zeros((1,8))
    res[0, 0] = layer_0
    res[0, 1] = layer_1
    res[0, 2] = layer_2
    res[0, 3] = rec_do
    res[0, 4] = main_do
    res[0, 5] = layer_end
    res[0, 6] = end_do
    res[0, 7] = np.round(np.min(history.history['val_loss']), decimals=3)
    
    del model
    K.clear_session()
    
    if grid_rep == 0:
        save_df = pd.DataFrame(res, columns=col_names)
    else:
        save_df = pd.read_csv('/scratch-shared/phil/LNP/LNP_data_09/LSTM_model_selection_' + fit_method + '_' + fold + '.csv')
        df = pd.DataFrame(res, columns=col_names)
        save_df = save_df.append(df, ignore_index=True, sort=False)
    
    save_df.to_csv('/scratch-shared/phil/LNP/LNP_data_09/LSTM_model_selection_' + fit_method  + '_' + fold + '.csv', index=False)